In [1]:
import numpy as np

In [6]:
landmarks_rt = np.load("demo/00181_aligned_rt.npy")
print(landmarks_rt.shape)
landmarks_rt

(838, 6)


array([[ 1.74477384e-01, -1.63831685e-01, -1.35869209e-01,
        -3.43874127e+00,  2.37755480e+01, -4.00866363e+01],
       [ 1.67067050e-01, -1.67639249e-01, -1.50782968e-01,
        -5.11945810e+00,  2.62284567e+01, -3.92865724e+01],
       [ 1.72185677e-01, -1.69576100e-01, -1.48127622e-01,
        -5.29403237e+00,  2.60699932e+01, -4.01751540e+01],
       ...,
       [ 2.02502671e-02,  1.24621935e-01, -5.38594353e-02,
        -1.73131726e+00,  5.37846570e+00,  1.13835737e+01],
       [ 2.43196351e-02,  1.48991006e-01, -5.70397839e-02,
        -1.70376107e+00,  5.34112077e+00,  1.53030257e+01],
       [ 2.62054949e-02,  1.42276164e-01, -5.33861444e-02,
        -2.31000922e+00,  5.23995696e+00,  1.49240854e+01]])

In [7]:
landmarks_front = np.load("demo/00181_aligned_front.npy")
print(landmarks_front.shape)
landmarks_front

(838, 68, 3)


array([[[ 71.65449107, 105.97002259, -58.03470724],
        [ 74.14772765, 122.49500343, -57.93752665],
        [ 77.78608195, 135.72322465, -58.12510086],
        ...,
        [130.86798778, 154.05465987,  31.62941615],
        [125.29169835, 154.66378422,  32.0362845 ],
        [122.16683297, 155.06906687,  31.92402122]],

       [[ 72.29225688, 104.91738109, -56.79290427],
        [ 75.12423307, 122.14263469, -56.51504764],
        [ 77.30651153, 134.77508331, -56.98304275],
        ...,
        [131.78641774, 153.83821056,  30.70624466],
        [126.1933492 , 154.51482319,  31.24115416],
        [122.33352098, 155.11386138,  30.8845388 ]],

       [[ 73.02702758, 106.36417218, -57.70124326],
        [ 74.00476878, 121.58658443, -58.17543856],
        [ 78.03596307, 136.30661331, -58.16969863],
        ...,
        [131.93041094, 154.8424242 ,  31.41045394],
        [126.35158949, 155.51581872,  31.83988219],
        [120.61611178, 154.03418035,  31.01805626]],

       ...,

      

In [8]:
landmarks = np.load("demo/00181_aligned.npy")
print(landmarks.shape)
landmarks_fr

(838, 68, 3)


array([[[ 58.5163335 ,  85.9147421 , -43.72132492],
        [ 58.5163335 , 102.35510907, -46.72382355],
        [ 60.08208273, 115.66397757, -49.67019272],
        ...,
        [122.71205215, 156.37345769,  25.36199188],
        [117.23192983, 156.37345769,  26.62683868],
        [114.10043135, 156.37345769,  26.99474907]],

       [[ 59.97498282,  84.04582134, -42.78192902],
        [ 59.97498282, 101.26906293, -45.63965607],
        [ 59.97498282, 113.79505681, -48.40725327],
        ...,
        [123.38782686, 155.28741155,  24.24307251],
        [117.90770453, 155.28741155,  25.65537834],
        [113.99333145, 155.28741155,  25.90433311]],

       [[ 60.73904299,  85.3516753 , -43.75348663],
        [ 59.17329375, 100.22629303, -46.78611755],
        [ 60.73904299, 115.10091077, -49.8232193 ],
        ...,
        [124.15188702, 156.59326551,  24.54066467],
        [118.6717647 , 156.59326551,  25.85572243],
        [113.19164238, 154.24464166,  26.32324791]],

       ...,

      

In [16]:
import os
import numpy as np
import torch
import cv2
from collections import OrderedDict
from PIL import Image
import pickle as pkl

from options.test_options import TestOptions
from data.data_loader import CreateDataLoader
from models.models import create_model
import util.util as util
from util.visualizer import Visualizer
from util import html

from tqdm import tqdm
import pdb

import warnings
warnings.simplefilter('ignore')

def add_audio(video_name, audio_dir):
    command = 'ffmpeg -i ' + video_name  + ' -i ' + audio_dir + ' -vcodec copy  -acodec copy -y  ' + video_name.replace('.mp4','.mov')

    print (command)
    os.system(command)

def image_to_video(sample_dir = None, video_name = None):
    command = 'ffmpeg -framerate 25  -i ' + sample_dir +  '/%05d.jpg -c:v libx264 -y -vf "pad=ceil(iw/2)*2:ceil(ih/2)*2"  ' + video_name 
    print (command)
    os.system(command)

def get_param(pickle_data, pick_id, opt, lrs_package=None):
    paths = pickle_data[pick_id]

    # check shot
    ref_nums = opt.ref_img_id.split(',')
    if opt.n_shot % len(ref_nums) != 0:
        print('reference number error')
        exit(0)
    else:
        ref_nums = ref_nums * (opt.n_shot // len(ref_nums))
        opt.ref_img_id = ','.join(ref_nums)

    if opt.dataset_name == 'vox':
        # target
        opt.tgt_video_path = paths[0]+"_aligned.mp4"
        if opt.no_head_motion:
            opt.tgt_lmarks_path = paths[0]+"_aligned_front.npy"
        else:
            opt.tgt_lmarks_path = paths[0]+"_aligned.npy"
        opt.tgt_rt_path = paths[0]+"_aligned_rt.npy"
        opt.tgt_ani_path = paths[0]+"_aligned_ani.mp4"

        # reference
        ref_paths = paths
        opt.ref_ani_id = int(paths[-1])
        opt.ref_front_path = ref_paths[0]+"_aligned_front.npy"
        opt.ref_video_path = opt.tgt_video_path
        opt.ref_lmarks_path = paths[0]+"_aligned.npy"
        opt.ref_rt_path = opt.tgt_rt_path
        if opt.no_head_motion:
            opt.ref_img_id = str(opt.ref_ani_id)
            opt.n_shot = 1

        audio_tgt_path = paths[0]+".wav"

    return audio_tgt_path

In [25]:
opt = TestOptions()

In [27]:
opt

In [26]:
### setup models
model = create_model(opt)
model.eval()

AttributeError: 'TestOptions' object has no attribute 'gpu_ids'